In [ ]:
# Functions below are not part of model pipeline

# Important! Filtering does not work optimally yet for each bat type.
# hyperparameter -30db
def filter_nonempty_max(
    slices,
    bat_min_freq,
    bat_max_freq,
    threshold_db=-30,
    # parameters for mel_freqs must be equal to preprocess parameters
    n_mels=512,
    fmin=17000,
    fmax=80000,
    save_dir=None,
) -> tuple[list, list]:

    # Convert expected bat frequency range to Mel bin indices
    mel_freqs = librosa.mel_frequencies(n_mels=n_mels, fmin=fmin, fmax=fmax)
    bat_bins = np.where((mel_freqs >= bat_min_freq) & (mel_freqs <= bat_max_freq))[0]

    filtered = []
    empty = []
    k = 0
    # Determine for each slice whether a bat call might be present or not
    for S in slices:
        if save_dir:
            savepath = os.path.join(save_dir, f"plot_{k}.png")
        else:
            savepath = None

        # Max energy in bat bins (more negative means quieter)
        max_energy = np.max(S[bat_bins, :])

        if max_energy > threshold_db:
            filtered.append(S)
            print(f"slice kept ({k})\nmax energy: {max_energy:.2f} dB")
            plot_spectrogram(
                S,
                path=savepath,
                title=f"kept, max, db: {max_energy:.2f}",
                hline=(bat_min_freq, bat_max_freq),
            )
        else:
            empty.append(S)
            print(f"slice deleted ({k})\nmax energy: {max_energy:.2f} dB")
            plot_spectrogram(
                S,
                path=savepath,
                title=f"deleted, max, db: {max_energy:.2f}",
                hline=(bat_min_freq, bat_max_freq),
            )
        k += 1
    return filtered, empty


# filtering seems to work for sample of myotis, pipistrellus,
# something wrong for brown long eared bat..?

In [ ]:
def plot_max_energy_histogram(
    slices,
    bat_min_freq,
    bat_max_freq,
    title="Histogram of Mean Energy in Bat Frequency Band",
    # parameters for mel_freqs must be equal to parameters used for preprocess function
    fmin=17000,
    fmax=80000,
    n_mels=512,
):
    # Get mel frequency bins
    mel_freqs = librosa.mel_frequencies(n_mels=n_mels, fmin=fmin, fmax=fmax)
    bat_bins = np.where((mel_freqs >= bat_min_freq) & (mel_freqs <= bat_max_freq))[0]

    # compute max energies
    energies = [np.max(S[bat_bins, :]) for S in slices]
    # Compute mean energy in the bat frequency bins for each slice

    # Plot histogram
    plt.figure(figsize=(8, 4))
    plt.hist(energies, bins=50, color="purple", edgecolor="black")
    plt.title(f"{title}")
    plt.xlabel("Mean Energy (dB)")
    plt.ylabel("Number of Slices")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_mean_energy_histogram(
    slices,
    # parameters for mel_freqs must be equal to parameters used for preprocess function
    fmin=17000,
    fmax=80000,
    n_mels=512,
    bat_min_freq=41000,
    bat_max_freq=48000,
    title="Histogram of Mean Energy in Bat Frequency Band",
):
    # Get mel frequency bins
    mel_freqs = librosa.mel_frequencies(n_mels=n_mels, fmin=fmin, fmax=fmax)
    bat_bins = np.where((mel_freqs >= bat_min_freq) & (mel_freqs <= bat_max_freq))[0]

    # Compute mean energy in the bat frequency bins for each slice
    mean_energies = [np.mean(S[bat_bins, :]) for S in slices]

    # Plot histogram
    plt.figure(figsize=(8, 4))
    plt.hist(mean_energies, bins=100, color="purple", edgecolor="black")
    plt.title(f"{title}")
    plt.xlabel("Mean Energy (dB)")
    plt.ylabel("Number of Slices")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return mean_energies